<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/004_Agents_CustomerServiceTriage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### **Customer Service Triage Agent** 🎧  
**Goal**: Build an agent that handles user support messages and routes them to the right department or tool (e.g., sales, refunds, tech).

#### 🔧 Tools:
- `route_to_sales()`
- `initiate_refund()`
- `log_support_ticket()`

#### 🧠 New Concepts:
- Tool delegation via natural language
- Simulating a decision tree with an agent
- Polite rejections and fallback handoffs

---

## 🧠 What Your Triage Agent Is Doing (Conceptually)

### 🎯 Goal:
Interpret incoming user messages and **route them to the correct team or action**, based on the **intent** behind the message.

This is a classic use case for AI agents: when messages are unpredictable, unstructured, and varied — but decisions must be made consistently.

---

## 🔁 The Agent Loop (Step-by-Step)

### 1. **Receives Input from a User**
```python
"Can I get a refund on my last order?"
```

### 2. **Uses an LLM to Decide What Kind of Message It Is**
You provide the model with:
- A list of possible **tool choices**
- A very clear **instructional prompt**
- **Few-shot examples** that show how to match message → tool

🧠 So the model is acting as a **classifier**, but it’s doing it through **language understanding** rather than rules or keyword matching.

---

### 3. **Selects the Right Tool (Action)**
From your tool list:
```python
tools = {
    "route_to_sales": ...,
    "initiate_refund": ...,
    "log_tech_support": ...,
    "log_unrecognized": ...
}
```

The agent maps each user input to one of these tools.

---

### 4. **Executes the Tool**
The tool receives the user’s original message and returns a response:

```python
tools["initiate_refund"]("Can I get a refund?")
→ "💸 Refund process started: 'Can I get a refund?'"
```

Each tool simulates what would happen in a real business system:
- Routing the ticket
- Triggering automation
- Saving to a queue
- Escalating an issue



In [ ]:
!pip install transformers --quiet

## 🧰 Step 1: Setup & Tool Functions

In [ ]:
# !pip install transformers --quiet
from transformers import pipeline

# Load instruction-following model
llm = pipeline("text2text-generation", model="google/flan-t5-base")

# --- Tools ---
def route_to_sales(message):
    return f"✅ Routed to Sales: '{message}'"

def initiate_refund(message):
    return f"💸 Refund process started: '{message}'"

def log_tech_support(message):
    return f"🛠️ Logged for Technical Support: '{message}'"

def log_unrecognized(message):
    return f"🤖 Could not categorize the message: '{message}'"

# Tool registry
tools = {
    "route_to_sales": route_to_sales,
    "initiate_refund": initiate_refund,
    "log_tech_support": log_tech_support,
    "log_unrecognized": log_unrecognized
}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


## 🧠 Step 2: Build the Triage Agent Loop

This is where the LLM will:
1. Read a customer message
2. Choose the correct department (via tool name)
3. Call the tool
4. Return the routed response



In [ ]:
def triage_agent(user_input):
    # Prompt the model to classify the request
    prompt = f"""
You are a customer service triage agent. Your job is to route each message to the appropriate department by choosing one of these tools:

- route_to_sales → For questions about product pricing, availability, quotes, or upgrades.
- initiate_refund → For refund requests, cancellations, or billing issues.
- log_tech_support → For technical problems, broken features, or login issues.
- log_unrecognized → If the message doesn't fit any category.

INSTRUCTIONS:
- Respond with ONLY the tool name (e.g., route_to_sales).
- Do NOT write full sentences.
- If uncertain, default to: log_unrecognized.

Examples:
- "I'd like a refund for my last order" → initiate_refund
- "Do you have bulk pricing for schools?" → route_to_sales
- "The login page won't load on my phone" → log_tech_support
- "What’s your favorite movie?" → log_unrecognized

Now classify this message:
"{user_input}"
Response:
"""

    model_output = llm(prompt, max_new_tokens=20)[0]["generated_text"].strip().lower()

    matched_tool = next((name for name in tools if name in model_output), None)
    if not matched_tool:
        matched_tool = "log_unrecognized"

    return tools[matched_tool](user_input)


### 🧪 Step 3: Test Cases

In [ ]:
print(triage_agent("Can I get a refund on my last order?"))
print(triage_agent("Do you have a discount for small businesses?"))
print(triage_agent("My dashboard is just showing a white screen."))
print(triage_agent("What's your favorite snack?"))


💸 Refund process started: 'Can I get a refund on my last order?'
✅ Routed to Sales: 'Do you have a discount for small businesses?'
🛠️ Logged for Technical Support: 'My dashboard is just showing a white screen.'
🛠️ Logged for Technical Support: 'What's your favorite snack?'



### 💡 Breakdown of Your Test Results

### ✅ Refund request correctly routed
```
💸 Refund process started: 'Can I get a refund on my last order?'
```
Perfect match — LLM understood the refund context and selected the right path.

---

### ✅ Sales inquiry correctly routed
```
✅ Routed to Sales: 'Do you have a discount for small businesses?'
```
Model recognized intent to discuss pricing/partnerships — a job for Sales.

---

### ✅ Technical issue correctly routed
```
🛠️ Logged for Technical Support: 'My dashboard is just showing a white screen.'
```
✅ LLM understands “white screen” as a technical error. Excellent.

---

### ⚠️ Model misrouted a personal question
```
🛠️ Logged for Technical Support: 'What's your favorite snack?'
```
This should have triggered:
```
🤖 Could not categorize the message...
```

But instead, the model guessed **log_tech_support** — likely because:
- It had no good match and defaulted to the first “logical-sounding” tool.
- The model didn’t return `log_unrecognized` because the prompt needs more examples or stronger emphasis.

---

## 🛠️ Want to Improve It?

To make it more accurate:
- Add more off-topic examples in the prompt
- Reinforce the fallback option more strongly: *“If not sure, ALWAYS say log_unrecognized”*




## 🤖 What Are Multi-Agent Handoffs?

It means one agent can **pass control** to another agent (or specialist) depending on the task.

For example:
- Your **triage agent** identifies a refund request
- Instead of handling it directly, it **hands off** the task to a **refund agent**
- The refund agent does the specialized work (e.g., validating the request, issuing a refund)

---

## 🧠 Why It Matters

| Traditional Agent | Multi-Agent System |
|-------------------|--------------------|
| Single LLM handles everything | Tasks are **delegated** to the best-suited agent |
| Monolithic, harder to scale | Modular and **composable** |
| Flat logic | Can represent **real business workflows** |

---

## 🛠️ Step 1: Define Specialized Sub-Agents

Let’s add 3 lightweight agents to your notebook:

```python
# Specialized refund agent
def refund_agent(user_input):
    return f"💸 Refund Agent: Processing refund request — '{user_input}'"

# Specialized sales agent
def sales_agent(user_input):
    return f"📦 Sales Agent: Preparing quote or offer — '{user_input}'"

# Specialized tech support agent
def tech_agent(user_input):
    return f"🛠️ Tech Support Agent: Logging issue and starting diagnosis — '{user_input}'"
```

---

## 🧩 Step 2: Update Your Routing Tools to Handoff

Now replace the tools in your original tool registry with these **handoffs**:

```python
tools = {
    "route_to_sales": sales_agent,
    "initiate_refund": refund_agent,
    "log_tech_support": tech_agent,
    "log_unrecognized": lambda msg: f"🤖 General Agent: I couldn’t route this request — '{msg}'"
}
```

## ✅ What You’ve Built

| Component | What It Does |
|----------|---------------|
| **Triage agent** | Classifies the request and routes it |
| **Sub-agents** | Handle specialized workflows |
| **Agent handoff** | Simulates real customer support teams with specialized LLM agents |
| **Composability** | You can now scale your system just by adding new agents |



In [ ]:
# Load instruction-following model
llm = pipeline("text2text-generation", model="google/flan-t5-base")

# --- Tools ---
def route_to_sales(message):
    return f"✅ Routed to Sales: '{message}'"

def initiate_refund(message):
    return f"💸 Refund process started: '{message}'"

def log_tech_support(message):
    return f"🛠️ Logged for Technical Support: '{message}'"

def log_unrecognized(message):
    return f"🤖 Could not categorize the message: '{message}'"

# Specialized refund agent
def refund_agent(user_input):
    return f"💸 Refund Agent: Processing refund request — '{user_input}'"

# Specialized sales agent
def sales_agent(user_input):
    return f"📦 Sales Agent: Preparing quote or offer — '{user_input}'"

# Specialized tech support agent
def tech_agent(user_input):
    return f"🛠️ Tech Support Agent: Logging issue and starting diagnosis — '{user_input}'"

tools = {
    "route_to_sales": sales_agent,
    "initiate_refund": refund_agent,
    "log_tech_support": tech_agent,
    "log_unrecognized": lambda msg: f"🤖 General Agent: I couldn’t route this request — '{msg}'"
}

def triage_agent(user_input):
    # Prompt the model to classify the request
    prompt = f"""
You are a customer service triage agent. Your job is to route each message to the appropriate department by choosing one of these tools:

- route_to_sales → For questions about product pricing, availability, quotes, or upgrades.
- initiate_refund → For refund requests, cancellations, or billing issues.
- log_tech_support → For technical problems, broken features, or login issues.
- log_unrecognized → If the message doesn't fit any category.

INSTRUCTIONS:
- Respond with ONLY the tool name (e.g., route_to_sales).
- Do NOT write full sentences.
- If uncertain, default to: log_unrecognized.

Examples:
- "I'd like a refund for my last order" → initiate_refund
- "Do you have bulk pricing for schools?" → route_to_sales
- "The login page won't load on my phone" → log_tech_support
- "What’s your favorite movie?" → log_unrecognized

Now classify this message:
"{user_input}"
Response:
"""

    model_output = llm(prompt, max_new_tokens=20)[0]["generated_text"].strip().lower()

    matched_tool = next((name for name in tools if name in model_output), None)
    if not matched_tool:
        matched_tool = "log_unrecognized"

    return tools[matched_tool](user_input)

print(triage_agent("Can I get a refund for my last order?"))
print(triage_agent("Do you offer a bundle discount?"))
print(triage_agent("My reports page is crashing."))
print(triage_agent("What's your favorite Marvel movie?"))

Device set to use cpu


💸 Refund Agent: Processing refund request — 'Can I get a refund for my last order?'
📦 Sales Agent: Preparing quote or offer — 'Do you offer a bundle discount?'
🛠️ Tech Support Agent: Logging issue and starting diagnosis — 'My reports page is crashing.'
🛠️ Tech Support Agent: Logging issue and starting diagnosis — 'What's your favorite Marvel movie?'


In [ ]:
# Load instruction-following model
# llm = pipeline("text2text-generation", model="google/flan-t5-base")

# --- Tools ---
def route_to_sales(message):
    return f"✅ Routed to Sales: '{message}'"

def initiate_refund(message):
    return f"💸 Refund process started: '{message}'"

def log_tech_support(message):
    return f"🛠️ Logged for Technical Support: '{message}'"

def log_unrecognized(message):
    return f"🤖 Could not categorize the message: '{message}'"

# Specialized refund agent
def refund_agent(user_input):
    return f"💸 Refund Agent: Processing refund request — '{user_input}'"

# Specialized sales agent
def sales_agent(user_input):
    return f"📦 Sales Agent: Preparing quote or offer — '{user_input}'"

# Specialized tech support agent
def tech_agent(user_input):
    return f"🛠️ Tech Support Agent: Logging issue and starting diagnosis — '{user_input}'"

tools = {
    "route_to_sales": sales_agent,
    "initiate_refund": refund_agent,
    "log_tech_support": tech_agent,
    "log_unrecognized": lambda msg: f"🤖 General Agent: I couldn’t route this request — '{msg}'",
    "logunrecognized": lambda msg: f"🤖 (alternate case caught) — '{msg}'"  # just in case
}


def triage_agent(user_input):
    # Prompt the model to classify the request
    prompt = f"""
You are a customer service triage agent. Your job is to route each message to the correct department by selecting one of the following tools:

- route_to_sales → Pricing, discounts, upgrades
- initiate_refund → Refunds, billing, cancellations
- log_tech_support → Bugs, app issues, access problems
- LOG_UNRECOGNIZED → Anything that doesn't belong in the above categories

INSTRUCTIONS:
- Return ONLY one of these tool names
- If unsure, always return LOG_UNRECOGNIZED

Examples:
- "I'd like a refund for my last order" → initiate_refund
- "Do you have bulk pricing for schools?" → route_to_sales
- "The login page won't load on my phone" → log_tech_support
- "What’s your favorite Marvel movie?" → LOG_UNRECOGNIZED
- "How’s your day going?" → LOG_UNRECOGNIZED
- "Tell me a joke" → LOG_UNRECOGNIZED

Now classify this message:
"{user_input}"
Response:
"""


    model_output = llm(prompt, max_new_tokens=20)[0]["generated_text"].strip().lower()

    matched_tool = next((name for name in tools if name in model_output.replace("_", "").lower()), None)
    if not matched_tool:
        matched_tool = "log_unrecognized"


    return tools[matched_tool](user_input)

print(triage_agent("Can I get a refund for my last order?"))
print(triage_agent("Do you offer a bundle discount?"))
print(triage_agent("My reports page is crashing."))
print(triage_agent("What's your favorite Marvel movie?"))

🤖 General Agent: I couldn’t route this request — 'Can I get a refund for my last order?'
🤖 General Agent: I couldn’t route this request — 'Do you offer a bundle discount?'
🤖 General Agent: I couldn’t route this request — 'My reports page is crashing.'
🤖 General Agent: I couldn’t route this request — 'What's your favorite Marvel movie?'


In [2]:
import json
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

notebook_path = "/content/drive/My Drive/AI AGENTS/004_Agents_CustomerServiceTriage.ipynb"

# Load the notebook JSON
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = json.load(f)

# 1. Remove widgets from notebook-level metadata
if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]
    print("✅ Removed notebook-level 'widgets' metadata.")

# 2. Remove widgets from each cell's metadata
for i, cell in enumerate(nb.get("cells", [])):
    if "metadata" in cell and "widgets" in cell["metadata"]:
        del cell["metadata"]["widgets"]
        print(f"✅ Removed 'widgets' from cell {i}")

# Save the cleaned notebook
with open(notebook_path, 'w', encoding='utf-8') as f:
    json.dump(nb, f, indent=2)

print("✅ Notebook deeply cleaned. Try uploading to GitHub again.")

Mounted at /content/drive
✅ Notebook deeply cleaned. Try uploading to GitHub again.


### **3. Blog Research Agent** 🔍  
**Goal**: Create an agent that takes a blog topic and does lightweight research, saving summaries for key subtopics.

#### 🔧 Tools:
- `search_web(query)` (mocked)
- `summarize_text(text)`
- `save_summary(topic, summary)`

#### 🧠 New Concepts:
- Multi-step workflows
- Agent-as-research-assistant
- LLM chain of reasoning (search → summarize → store)